<a href="https://colab.research.google.com/github/ahmadsaad007/Deep_Learning_Projects/blob/master/Image_Classification_Using_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from keras.models import Sequential
import os

In [0]:
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import *
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.layers import Input

In [7]:
try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))

except KeyError:
  print('TPU not found')

Found TPU at: grpc://10.23.241.170:8470


In [9]:
"""device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))"""

"device_name = tf.test.gpu_device_name()\nif device_name != '/device:GPU:0':\n  raise SystemError('GPU device not found')\nprint('Found GPU at: {}'.format(device_name))"

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [13]:
train_path = "/content/drive/My Drive/Eye Disease Research/train"
test_path = "/content/drive/My Drive/Eye Disease Research/test"

imgdata = ImageDataGenerator( rescale=1.0/255.0, validation_split=0.20)
train_batches = imgdata.flow_from_directory(train_path,class_mode='categorical', subset='training', target_size=(768, 496) ,
                                            classes = ['CNV','DME','DRUSEN','NORMAL'], batch_size = 16)
test_batches = imgdata.flow_from_directory(test_path, target_size=(768, 496) ,class_mode='categorical',
                                           classes = ['CNV','DME','DRUSEN','NORMAL'], batch_size = 16)
valid_batches = imgdata.flow_from_directory(train_path, subset='validation', target_size=(768, 496) ,class_mode='categorical',
                                            classes = ['CNV','DME','DRUSEN','NORMAL'], batch_size = 16)


Found 66800 images belonging to 4 classes.
Found 1001 images belonging to 4 classes.
Found 16699 images belonging to 4 classes.


In [19]:
"""tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)
"""
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

"""tf.config.experimental_connect_to_host('grpc://' + os.environ['COLAB_TPU_ADDR'])
resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)"""

Running on TPU  ['10.23.241.170:8470']


INFO:tensorflow:Initializing the TPU system: grpc://10.23.241.170:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.23.241.170:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


"tf.config.experimental_connect_to_host('grpc://' + os.environ['COLAB_TPU_ADDR'])\nresolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])\ntf.tpu.experimental.initialize_tpu_system(resolver)\nstrategy = tf.distribute.experimental.TPUStrategy(resolver)"

In [0]:
def model_create():
  vgg16_model = VGG16(input_tensor=Input(shape=(768, 496, 3)),input_shape=(768,496,3), include_top = False)
  model= Sequential()
  model.add(vgg16_model)
 
  #Make the vgg16's layers non-trainable
  for layer in model.layers:
    layer.trainable = False
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(4, activation='softmax'))
  return model

In [0]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(train_batches.classes), 
                                                  train_batches.classes)

In [0]:
lr = [0.0005, 0.001,0.005, 0.01, 0.05, 0.1]
for i in lr:
  print("Learning Rate: " + str(i))
  with strategy.scope():
    model = model_create()
    model.compile(Adam(learning_rate=i), loss='categorical_crossentropy', 
              metrics=['accuracy'])
    
  model.fit(train_batches, steps_per_epoch = 512, epochs = 10, 
            validation_data = valid_batches, validation_steps = 64, 
            class_weight = class_weights, shuffle = True)

Learning Rate: 0.0005
Epoch 1/10
512/512 [==============================] - 2102s 4s/step - loss: 1.1531 - accuracy: 0.6777 - val_loss: 0.6031 - val_accuracy: 0.7520
Epoch 2/10
326/512 [==================>...........] - ETA: 10:25 - loss: 0.6198 - accuracy: 0.7801

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 3357 could not be retrieved. It could be because a worker has died.
  UserWarning)


512/512 [==============================] - 2049s 4s/step - loss: 0.5855 - accuracy: 0.7911 - val_loss: 0.5620 - val_accuracy: 0.8340
Epoch 3/10
512/512 [==============================] - 1995s 4s/step - loss: 0.4621 - accuracy: 0.8330 - val_loss: 0.4639 - val_accuracy: 0.8467
Epoch 4/10
512/512 [==============================] - 2065s 4s/step - loss: 0.4551 - accuracy: 0.8345 - val_loss: 0.8936 - val_accuracy: 0.8311
Epoch 5/10
512/512 [==============================] - 2217s 4s/step - loss: 0.4346 - accuracy: 0.8409 - val_loss: 0.2387 - val_accuracy: 0.8193
Epoch 6/10
512/512 [==============================] - 2152s 4s/step - loss: 0.4112 - accuracy: 0.8535 - val_loss: 0.3307 - val_accuracy: 0.8545
Epoch 7/10
512/512 [==============================] - 2003s 4s/step - loss: 0.4028 - accuracy: 0.8524 - val_loss: 0.1979 - val_accuracy: 0.8242
Epoch 8/10
512/512 [==============================] - 2109s 4s/step - loss: 0.3794 - accuracy: 0.8645 - val_loss: 0.4717 - val_accuracy: 0.7812
Epo